In [10]:
import os
import zipfile
import urllib.request as ur
import shutil
import re

import pandas as pd
import pickle
import subprocess
import numpy as np

In [2]:
URL_BASE = 'https://dadosabertos.rfb.gov.br/CNPJ'
ZIP_PATH = 'zip'

In [4]:
# resource - extract information from web site
fp = ur.urlopen(URL_BASE)
f_btyes = fp.read()
file_str = f_btyes.decode("utf8")

fp.close()

In [5]:
# extracting link to download files
query = 'Estabelecimentos'
compile_link = re.compile(r'href=[\'"]?([^\'" >]+)')
companies = list(filter(lambda file: str(file).startswith(query), compile_link.findall(file_str)))

In [6]:
companies

['Estabelecimentos0.zip',
 'Estabelecimentos1.zip',
 'Estabelecimentos2.zip',
 'Estabelecimentos3.zip',
 'Estabelecimentos4.zip',
 'Estabelecimentos5.zip',
 'Estabelecimentos6.zip',
 'Estabelecimentos7.zip',
 'Estabelecimentos8.zip',
 'Estabelecimentos9.zip']

In [30]:
def file_len(fname):
    p = subprocess.Popen(['wc', '-l', fname], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    result, err = p.communicate()
    
    if p.returncode != 0:
        raise IOError(err)
    
    return int(result.strip().split()[0])

In [9]:
# download resource from web site
if not os.path.exists(ZIP_PATH):
        os.mkdir(ZIP_PATH)

if not os.path.exists(f'{ZIP_PATH}/{str(companies[0]).lower()}'):
    # download zip file
    ur.urlretrieve(f'{URL_BASE}/{companies[0]}', companies[0])
    shutil.move(companies[0], f'{ZIP_PATH}')

# remove folder if exist
filename = companies[0].split('.')[0].lower()

if not os.path.exists(f'{ZIP_PATH}/{filename}.csv'):
    # unzip the download file
    zip_ref = zipfile.ZipFile(f'{ZIP_PATH}/{companies[0]}', 'r')
    zip_name = zip_ref.namelist()[0]

    # extract all file
    zip_ref.extractall(f'{ZIP_PATH}')

    # rename file
    os.rename(f'{ZIP_PATH}/{zip_name}', f'{ZIP_PATH}/{filename}.csv')
    
zip_ref.close()

In [23]:

names = [' cnpj_raiz ', 'cnpj_ordem ', 'cnpj_digito_verificador ', 'tipo ', 'nome_fantasia ', 'situacao_cadastral ', 'data_situacao_cadastral ', 'col8 ', 'col9 ', 'col10 ', 'data_inicio_atividade ', 'atividade_principal ', 'atividade_secundaria ', 'tipo_logradouro ', 'logradouro', 'numero ', 'complemento', 'bairro',   'cep      ', 'estado ', 'siafi_id ', 'ddd1 ', 'telefone1 ', 'ddd2 ', 'telefone2 ', 'ddd_fax ', 'fax      ', 'email                     ', 'situacao_especial ', 'data_situacao_especial ',]

In [25]:
df = pd.read_csv(f'{ZIP_PATH}/{filename}.csv', 
                 sep=';', 
                 dtype=str, 
                 encoding='latin1', 
                 chunksize=400000,
                 names=names,
                 low_memory=False)

In [26]:
for i, chunk in enumerate(df):
    with open(f'{ZIP_PATH}/{filename}{i+1}.pkl', "wb") as f:
        pickle.dump(chunk, f, pickle.HIGHEST_PROTOCOL)

KeyboardInterrupt: 

In [27]:
df = pd.read_pickle(f'{ZIP_PATH}/{filename}1.pkl')

In [28]:
df

,cnpj_raiz,cnpj_ordem,cnpj_digito_verificador,tipo,nome_fantasia,situacao_cadastral,data_situacao_cadastral,col8,col9,col10,...,siafi_id,ddd1,telefone1,ddd2,telefone2,ddd_fax,fax,email,situacao_especial,data_situacao_especial
0,20857537,0001,80,1,JULIANA MODAS,08,20170509,01,NaN,NaN,...,8569,51,80190160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20857551,0001,83,1,RAFAEL SAKAI,08,20150114,01,NaN,NaN,...,9073,67,92956332,NaN,NaN,NaN,NaN,sakai@globo.com,NaN,NaN
2,20857565,0001,05,1,EDCARLOS MENDES,08,20180201,67,NaN,NaN,...,9167,65,92180205,NaN,NaN,NaN,NaN,edcarlos.mendes@hotmail.com,NaN,NaN
3,20857575,0001,32,1,COMERCIAL BATISTA,08,20160517,01,NaN,NaN,...,0130,99,36231077,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20857589,0001,56,1,PRO-DENTE,08,20160217,01,NaN,NaN,...,5241,34,36711525,34,88991525,NaN,NaN,CONTABJW@MSN.COM,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,25655579,0001,51,1,NaN,08,20161231,01,NaN,NaN,...,6619,0,0,NaN,NaN,11,27090194,pr22jundiai@gmail.com,NaN,NaN
399996,25655591,0001,66,1,NaN,08,20161231,01,NaN,NaN,...,4359,0,0,NaN,NaN,31,37313877,vasconcelosthiago3@gmail.com,NaN,NaN
399997,25655603,0001,52,1,NaN,08,20161231,01,NaN,NaN,...,4517,0,0,NaN,NaN,31,38631206,contgol@gmail.com,NaN,NaN
399998,25655615,0001,87,1,NaN,08,20161231,01,NaN,NaN,...,7097,0,0,NaN,NaN,17,33048495,valdenor10@hotmail.com,NaN,NaN
